# cli

> Fill in a module description here

In [ ]:
# | default_exp cli

In [ ]:
# | export
import code_tokenizers
import os
import shutil
import urllib.request

from fastcore.script import *
from pathlib import Path
from tree_sitter import Language

In [ ]:
# | hide # fmt: skip
from nbdev.showdoc import *

In [ ]:
# | export
_GRAMMARs = {
    "python": "https://github.com/tree-sitter/tree-sitter-python/archive/refs/tags/v0.20.0.zip",
}

In [ ]:
# | export
def _download_grammars(languages):
    """
    Download the tree-sitter grammars for the specified languages.

    If the languages argument is the string 'all', all available grammars will be downloaded.
    Otherwise, the argument should be a list of language codes to download.
    """
    try:
        grammars = (
            _GRAMMARs if languages == "all" else {k: _GRAMMARs[k] for k in languages}
        )
    except KeyError as e:
        raise ValueError(
            f"Invalid or unsupported language: {e}. Supported languages: {list(_GRAMMARs.keys())}"
        )

    langs = []
    grammar_dir = Path(code_tokenizers.__file__).parent / "grammars"
    grammar_dir.mkdir(exist_ok=True)
    for name, url in grammars.items():
        repo_dir = grammar_dir / name
        if not repo_dir.exists():
            # Download the tagged archive
            urllib.request.urlretrieve(url, f"{repo_dir}.zip")
            # Unzip the repository archive and remove the zip file
            shutil.unpack_archive(f"{repo_dir}.zip", repo_dir)
            os.remove(f"{repo_dir}.zip")
            ts_path = list(repo_dir.iterdir())[0]
            # Move the contents of the tagged archive to the repo directory
            for f in ts_path.iterdir():
                shutil.move(f, repo_dir)

        langs.append(str(repo_dir))

    Language.build_library(
        # Store the library in the directory
        str(grammar_dir / "tree-sitter-languages.so"),
        # Include one or more languages
        langs,
    )

In [ ]:
# test the above method
_download_grammars("all")

grammar_dir = Path(code_tokenizers.__file__).parent / "grammars"
for name, _ in _GRAMMARs.items():
    repo_dir = grammar_dir / name
    assert repo_dir.exists()

assert (grammar_dir / "tree-sitter-languages.so").exists()

In [ ]:
# | export
@call_parse
def download_grammars(
    languages: Param("Languages to download", str, nargs="+") = "all",
):
    """Download Tree-sitter grammars"""
    _download_grammars(languages)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()